## train 데이타와 일반대화 데이타를 merge하여 하나의 파일로 만든다.

0. 파일 
  - original_train_data : train_no_dup.csv
  - 일반대화 데이타 : conversation_normal.csv
1. train 데이타 내용 확인
2. 중복 데이타와 null 데이타 확인
3. original train과 일반 대화 데이타 merge
4. merged train 데이타 csv 파일 저장
5. validation용으로 나누어 별도 csv 파일 저장

### 1. train 데이타 내용 확인

In [1]:
import os
import pandas as pd
import tensorflow
import numpy as np

In [2]:
orig_train_path ="../data/raw_csv/train_no_dup.csv"
orig_train = pd.read_csv(orig_train_path)
orig_train.head()

,idx,class,conversation
0,3748,직장 내 괴롭힘 대화,지은씨 커피 좀 사와요.\n그건 싫어요.\n사오라면 사와야지 인턴주제.\n그러니까요...
1,3149,직장 내 괴롭힘 대화,자 점심식사들 합시다\n 와 점심시간이다. 과장님 밥 어디서 먹죠 오늘은?\n 대리...
2,1866,협박 대화,어이 이쁜이 시간이 지났는데 약속을 안지키네\n네.엄마가 아프셔서.\n그래 엄마가 ...
3,1493,기타 괴롭힘 대화,우리 귀여운 철수야 여기 돈 줄테니까 핫도그랑 콜라좀 사다줄래? 남은 돈은 너 가져...
4,366,직장 내 괴롭힘 대화,어이 김대리\n예 이부장님\n오늘 하라고 했던 서류 정리 다 끝냈어?\n아.그게 아...


In [3]:
class_counts = orig_train['class'].value_counts()

print(class_counts)

orig_train.shape

기타 괴롭힘 대화      1011
갈취 대화           973
직장 내 괴롭힘 대화     970
협박 대화           892
Name: class, dtype: int64


(3846, 3)

In [4]:
# 일반 대화
normal_data_path ="../data/raw_csv/conversation_normal.csv"
normal_data = pd.read_csv(normal_data_path)
normal_data.head()

,idx,class,conversation
0,4001,일반 대화,"집에서 자주 만드는 음식 있어?\n응, 김치찌개 자주 만들어.\n오, 김치찌개! 어..."
1,4002,일반 대화,"너는 어떤 장르 좋아해?\n나는 록 음악을 좋아해.\n오, 록! 그럼 Queen 좋..."
2,4003,일반 대화,"오늘 점심 뭐 먹을까요?\n저는 김밥 먹고 싶은데, 다른 사람들은 어떻게 생각해요?..."
3,4004,일반 대화,과제 다 했어?\n아직이야. 너무 많아.\n나도 겨우 끝냈어.\n우리 같이 도서관 갈래?
4,4005,일반 대화,"이 장난감은 여러 가지 색상 중에서 선택할 수 있나요?\n네, 다양한 색상이 있어서..."


In [5]:
normal_data.shape

(932, 3)

### 2. 일반 대화 중복 데이타와 null 데이타 확인

In [6]:
print('normal_data의 전체 샘플수 :', (len(normal_data)))
print('normal_data의 conversation 열에서 중복을 배제한 유일한 샘플의 수 :', normal_data['conversation'].nunique())
print('normal_data의 중복제거 샘플수 :',(len(normal_data) - normal_data['conversation'].nunique()) )
print(normal_data.isnull().sum())

normal_data의 전체 샘플수 : 932
normal_data의 conversation 열에서 중복을 배제한 유일한 샘플의 수 : 791
normal_data의 중복제거 샘플수 : 141
idx             0
class           0
conversation    0
dtype: int64


In [7]:
normal_data.drop_duplicates(subset = ['conversation'], inplace=True)
print('normal_data의 중복 제거한 전체 샘플수 :', (len(normal_data)))

normal_data의 중복 제거한 전체 샘플수 : 791


### 3. Original Train과 일반대화 데이타 merge

In [8]:
train_data = pd.concat([orig_train, normal_data], ignore_index=True)


In [9]:
train_data = train_data.sample(frac=1).reset_index(drop=True)

In [10]:
class_counts = train_data['class'].value_counts()

print(class_counts)

기타 괴롭힘 대화      1011
갈취 대화           973
직장 내 괴롭힘 대화     970
협박 대화           892
일반 대화           791
Name: class, dtype: int64


In [11]:
train_data[train_data['class']=='일반 대화'][:3]

,idx,class,conversation
1,5078,일반 대화,요즘 읽고 있는 책 있어?\n그거 좋은 생각이야.\n좋은 시간 보냈네.\n조언 고마...
10,4691,일반 대화,최근에 어떤 취미를 시작했어? 나는 사진 찍는 걸 배우고 있어. 카메라를 새로 샀고...
15,4705,일반 대화,주말에 뭐 할 계획이야? 친구들과 함께 영화를 보러 가기로 했어. 어떤 영화 볼 건...


#### 4. merged_train 데이타 csv 파일로 저장

- ../data/raw_csv/merged_train.csv

In [12]:
train_data.to_csv('../data/raw_csv/merged_train.csv', index=False, encoding='utf-8-sig')  # index=False는 인덱스를 저장하지 않도록 함

In [13]:
train_data_path ="../data/raw_csv/merged_train.csv"
train_data = pd.read_csv(train_data_path)
train_data.head()

,idx,class,conversation
0,1516,갈취 대화,어이 가방 좋은데!?\n네!? 왜 그러시죠\n아니 우리가 밥을 굶어서 편의점 라면이...
1,5078,일반 대화,요즘 읽고 있는 책 있어?\n그거 좋은 생각이야.\n좋은 시간 보냈네.\n조언 고마...
2,350,협박 대화,간도 커 어딜 튀려고 !\n아니 미주엄마 내가 딱 일주일만 더 쓰고 주려고 했는데 ...
3,3247,기타 괴롭힘 대화,야 너 뭐하냐\n공부해\n뭔공부\n토익공부\n그런다고 점수 올릴 수 있겠니?\n왜 ...
4,3511,협박 대화,아저씨 제발 살려주세요. 칼만 좀 내려놔주시면.\n소리내지마. 소리 내면 찌른다.\...


In [14]:
train_data.shape

(4637, 3)

##5. train data와 validation data로 나눠서 저장한다. 

In [18]:
from sklearn.model_selection import train_test_split
random_state=42

train_df, val_df = train_test_split(train_data, test_size=0.2, 
                                    random_state=random_state,shuffle=True, stratify=train_data['class'])

print("훈련 수:", len(train_df))
print("검증 수:", len(val_df))

훈련 수: 3709
검증 수: 928


In [23]:
train_df.head()

,idx,class,conversation
1288,673,직장 내 괴롭힘 대화,김과장 라인 잘 타. 누가 회사 이끄는 건데.\n 아무래도 이건 부당한 것 같습니다...
1684,243,협박 대화,이게 어떻게 된거야.? 엄마 수술은 어떻게 된거냐고\n 엄마가 이제 괜찮다고 해서 ...
209,2006,갈취 대화,이사님 이 사진. 모르지 않으실텐데요?\n이. 이건. \n이사님이 여기까지 오신거 ...
1707,1073,협박 대화,너 지금 나랑 헤어지자고 한거야? 죽고싶어?\n 그만하자\n 너 내가 헤어지자고 말...
3163,893,기타 괴롭힘 대화,철수 씨는 그 학교 나와서 그 연봉 받고 살만해요?\n그게 무슨 말씀이시죠?\n아니...


In [22]:
val_df.head()

,idx,class,conversation
3941,549,갈취 대화,김사장님 별일 없으세요 ?\n자네가 여긴 어쩐 일인가.\n아니 말씀을 참 섭섭하게 ...
3583,2111,기타 괴롭힘 대화,눈알은 장식이니? 너 장님이야?\n미안해.\n미안하면 다야? 사람 쳤으면 사과 부터...
3048,3257,직장 내 괴롭힘 대화,출산일이 언제라고? \n과장님 3달 남았어요\n복직은 언제 할건데? \n네? 출산휴...
3157,793,협박 대화,내가 술 좀 그만 먹으라고 했지? 죽을래?\n술을 먹든 말든 신경 꺼\n네가 술 먹...
3986,915,기타 괴롭힘 대화,야 이따 정답 알려줘\n안돼\n알려달라고\n안된다고\n너 따 당하고 싶냐?\n아니....


In [24]:
class_counts = train_df['class'].value_counts()
print(class_counts)

기타 괴롭힘 대화      809
갈취 대화          778
직장 내 괴롭힘 대화    776
협박 대화          713
일반 대화          633
Name: class, dtype: int64


In [25]:
class_counts = val_df['class'].value_counts()
print(class_counts)

기타 괴롭힘 대화      202
갈취 대화          195
직장 내 괴롭힘 대화    194
협박 대화          179
일반 대화          158
Name: class, dtype: int64


In [19]:
train_df.to_csv('../data/raw_csv/merged_train_train.csv', index=False, encoding='utf-8-sig')
val_df.to_csv('../data/raw_csv/merged_train_val.csv', index=False, encoding='utf-8-sig')

In [20]:
data_path ="../data/raw_csv/merged_train_train.csv"
df = pd.read_csv(data_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3709 entries, 0 to 3708
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   idx           3709 non-null   int64 
 1   class         3709 non-null   object
 2   conversation  3709 non-null   object
dtypes: int64(1), object(2)
memory usage: 87.1+ KB


In [21]:
data_path ="../data/raw_csv/merged_train_val.csv"
df = pd.read_csv(data_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 928 entries, 0 to 927
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   idx           928 non-null    int64 
 1   class         928 non-null    object
 2   conversation  928 non-null    object
dtypes: int64(1), object(2)
memory usage: 21.9+ KB
